In [54]:
import os
from math import *
from time import perf_counter_ns
import gc
os.chdir('tsp_dataset')

# Parse input 

In [51]:
def parse(fileaddress):
    CoOrdinates = {}
    with open(fileaddress) as f:
        lines = f.readlines()
    lines[0] = ': '.join(lines[0].split(' : '))
    Name = lines[0].split(" ")[1].strip()
    lines[4] = ': '.join(lines[4].split(' : '))
    EWT = lines[4].split(" ")[1].strip()
    lines[3] = ': '.join(lines[3].split(' : '))
    Dimension = lines[3].split(" ")[1].strip()
    flag = False
    for line in lines:
        line = ' '.join(line.split())
        if line.strip().split(" ")[0] == '1':
            flag = True
        elif flag == False:
            continue
        if line.strip().split(" ")[0] == 'EOF':
            break
        n,x,y = line.strip().split(" ")
        CoOrdinates[n] = (x,y)
    return Name, EWT, Dimension, CoOrdinates

<p>we save the coordinates in a dictionary with the vertex ID as the key and a (x,y) tuple as the data</p>

In [36]:
NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse('berlin52.tsp')
print(f"Data set name: {NAME}\nType: {EDGE_WEIGHT_TYPE}\nDimension: {DIMENSION}\nCoordinates: {Coordinates}")

Data set name: berlin52
Type: EUC_2D
Dimension: 52
Coordinates: {'1': ('565.0', '575.0'), '2': ('25.0', '185.0'), '3': ('345.0', '750.0'), '4': ('945.0', '685.0'), '5': ('845.0', '655.0'), '6': ('880.0', '660.0'), '7': ('25.0', '230.0'), '8': ('525.0', '1000.0'), '9': ('580.0', '1175.0'), '10': ('650.0', '1130.0'), '11': ('1605.0', '620.0'), '12': ('1220.0', '580.0'), '13': ('1465.0', '200.0'), '14': ('1530.0', '5.0'), '15': ('845.0', '680.0'), '16': ('725.0', '370.0'), '17': ('145.0', '665.0'), '18': ('415.0', '635.0'), '19': ('510.0', '875.0'), '20': ('560.0', '365.0'), '21': ('300.0', '465.0'), '22': ('520.0', '585.0'), '23': ('480.0', '415.0'), '24': ('835.0', '625.0'), '25': ('975.0', '580.0'), '26': ('1215.0', '245.0'), '27': ('1320.0', '315.0'), '28': ('1250.0', '400.0'), '29': ('660.0', '180.0'), '30': ('410.0', '250.0'), '31': ('420.0', '555.0'), '32': ('575.0', '665.0'), '33': ('1150.0', '1160.0'), '34': ('700.0', '580.0'), '35': ('685.0', '595.0'), '36': ('685.0', '610.0'), 

In [37]:
def computeWeight(First, Second, Type):
    x1 = float(First[0])
    x2 = float(Second[0])
    y1 = float(First[1])
    y2 = float(Second[1])
    if Type == "EUC_2D":
        return sqrt((x1-x2)**2 + (y1-y2)**2)
    if Type == "GEO":
        Pi = 3.141592
        R = 6378.388
        lon1 = (x1/180)*Pi
        lon2 = (x2/180)*Pi
        lat1 = (y1/180)*Pi
        lat2 = (y2/180)*Pi
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c


In [38]:
print(Coordinates['1'],Coordinates['2'])
print(computeWeight(Coordinates['1'],Coordinates['2'],EDGE_WEIGHT_TYPE))

('565.0', '575.0') ('25.0', '185.0')
666.1080993352356


In [39]:
def getKey(sdict):
    key_list = list(sdict.keys())
    return key_list

def makeGraph(Coordinates):
    keys = getKey(Coordinates)
    matrix = []
    for u in keys:
        for v in keys:
            if v == u:
                continue
            w = computeWeight(Coordinates[u],Coordinates[v],EDGE_WEIGHT_TYPE) # weight computed for edge(u,v)
            matrix.append((u,v,w))
    return keys, matrix

In [40]:
# Graph object
class Graph:
    def __init__(self, V, E, num_V, num_E):
        self.V = V
        self.E = E
        self.num_V = num_V
        self.num_E = num_E

In [41]:
class Kruskal_Efficient:
    
    def __init__(self, graph):
        self.graph = graph
        self.sets = {} # set of vertices
        self.MST = [] # Minimum Spanning Tree
    
    
    # make a set of vertices
    def make_sets(self):
        for v in self.graph.V:
            self.sets[v] = [v]
        
    
    
    # union the subsets which the vertices are not in the same sets
    def union(self, u_prnt, v_prnt):
        # get the size of two elements and append the vertices to the bigger one
        if (len(self.sets.get(u_prnt)) >= len(self.sets.get(v_prnt))):
            self.sets[u_prnt].extend(self.sets[v_prnt])
            self.sets.pop(v_prnt)
        
        # append the list of vertices of parent u to v
        else:
            self.sets[v_prnt].extend(self.sets[u_prnt])
            self.sets.pop(u_prnt)
    
    
    
    # find the parent of u and v vertices and return the parents
    def find_parent(self, u, v, items):
        u_key = v_key = 0
        for item in list(items):
            # item[0] is the key in dictionary
            # item[1] is the values in the dictionary
            key, value = item[0], item[1] 
            #if_true = all(x in value for x in[u, v]) # check if both u and v are in the same set
            # check the vertices in the value list and return the key as the parent of the vertex
            if u in value:
                u_key = item[0]
            if v in value:
                v_key = item[0]
            if u_key and v_key:
                break
        return (u_key, v_key)
    
    
    
    # make the MST tree
    def execute(self):
#         mst_weight = 0
#         lenv = len(self.graph.V) # number of vertices
        
        # sorting the edges based on the wight of the edges    
        E = sorted(self.graph.E, key = lambda m: m[2])
#         print("this: ", E)
        
        self.make_sets() # make a set of vertices
        # make a list of sets of key and value pairs to iterate through them
        items = self.sets.items()
        for e in E:
            # check if number of edges in MST are less than  nodes are 
            if((len(self.MST)+1) <  int(self.graph.num_E)):
                u, v, w = e
                u_parent, v_parent = self.find_parent(u, v, items)
                # if the vertices(u,v) are not in the same sets
                if (u_parent != v_parent):
                    self.union(u_parent, v_parent)
                    # add the edge to the MST[]
                    self.MST.append(e)
            # if the MST is completed, stop looping through the edges
            else:
                break
        return self.MST

    
    # calculate the final weight of the MST
    def MSTweight_EK(self):
        sum = 0
        for (u ,v, w) in self.MST:
            sum = sum + w
        return sum

In [42]:
graph = Graph(vertices, edges, num_V_E[0], num_V_E[1])
algo = Kruskal_Efficient(graph)
result = algo.execute()
print(result)
print("\nThe Weight of MST is: ", algo.MSTweight_EK())

[('35', '36', 15.0), ('24', '48', 15.811388300841896), ('34', '35', 21.213203435596427), ('5', '15', 25.0), ('5', '24', 31.622776601683793), ('5', '6', 35.35533905932738), ('20', '50', 35.35533905932738), ('38', '40', 35.35533905932738), ('37', '40', 41.23105625617661), ('39', '40', 42.720018726587654), ('36', '39', 43.01162633521314), ('24', '38', 44.721359549995796), ('2', '7', 45.0), ('1', '22', 46.09772228646444), ('32', '49', 50.0), ('1', '49', 64.03124237432849), ('8', '41', 64.03124237432849), ('4', '6', 69.6419413859206), ('19', '45', 75.0), ('7', '42', 76.15773105863909), ('34', '44', 80.0), ('18', '31', 80.15609770940699), ('36', '49', 81.39410298049853), ('9', '10', 83.21658488546619), ('19', '41', 91.92388155425118), ('20', '23', 94.33981132056604), ('22', '31', 104.4030650891055), ('4', '25', 109.20164833920778), ('27', '28', 110.11357772772621), ('46', '48', 125.0), ('26', '27', 126.19429464123962), ('16', '50', 130.38404810405297), ('16', '44', 132.38202294873727), ('3',

In [55]:
twoAppSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates = parse(file)
    vertices, edges = makeGraph(Coordinates)
    num_V_E = [len(vertices), len(edges)]
    gc.disable()
    start_time = perf_counter_ns()
    graph = Graph(vertices, edges, num_V_E[0], num_V_E[1])
    algo = Kruskal_Efficient(graph)
    print(NAME)
    result = algo.execute()
    weight = algo.MSTweight_EK()
    end_time = perf_counter_ns()
    time = end_time - start_time
    gc.enable()
    twoAppSol.append((NAME,weight,time))

berlin52
burma14
ch150
d493
dsj1000
eil51
gr202
gr229
kroA100
kroD100
pcb442
ulysses16.tsp
ulysses22.tsp


In [56]:
twoAppSol

[('berlin52', 6081.630541640883, 27753400),
 ('burma14', 830.7778546003948, 1594700),
 ('ch150', 5880.955830859872, 185165100),
 ('d493', 29284.55098300803, 4517393700),
 ('dsj1000', 15905257.207705617, 40049387900),
 ('eil51', 376.4905590196479, 64531700),
 ('gr202', 41918.8554509152, 430780200),
 ('gr229', 92987.17687347162, 508141800),
 ('kroA100', 18772.173204200863, 67838900),
 ('kroD100', 18596.715640086226, 66710800),
 ('pcb442', 46362.39053165435, 2941092600),
 ('ulysses16.tsp', 5268.036033544723, 12100400),
 ('ulysses22.tsp', 5429.474859481939, 1947600)]